Notes by - Kiran A Bendigeri
Please Read 'Read me' file.

Movie Recommendation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import sqrt
from scipy.stats.stats import pearsonr

movie_user_preferences = {'Jill': {'Avenger: Age of Ultron': 7.0,
                                   'Django Unchained': 6.5,
                                   'Gone Girl': 9.0,
                                   'Kill the Messenger': 8.0},
                          'Julia': {'Avenger: Age of Ultron': 10.0,
                                    'Django Unchained': 6.0,
                                    'Gone Girl': 6.5,
                                    'Kill the Messenger': 6.0,
                                    'Zoolander': 6.5},
                          'Max': {'Avenger: Age of Ultron': 7.0,
                                  'Django Unchained': 7.0,
                                  'Gone Girl': 10.0,
                                  'Horrible Bosses 2': 6.0,
                                  'Kill the Messenger': 5.0,
                                  'Zoolander': 10.0},
                          'Robert': {'Avenger: Age of Ultron': 8.0,
                                     'Django Unchained': 7.0,
                                     'Horrible Bosses 2': 5.0,
                                     'Kill the Messenger': 9.0,
                                     'Zoolander': 9.0},
                          'Sam': {'Avenger: Age of Ultron': 10.0,
                                  'Django Unchained': 7.5,
                                  'Gone Girl': 6.0,
                                  'Horrible Bosses 2': 3.0,
                                  'Kill the Messenger': 5.5,
                                  'Zoolander': 7.0},
                          'Toby': {'Avenger: Age of Ultron': 8.5,
                                   'Django Unchained': 9.0,
                                   'Zoolander': 2.0},
                          'William': {'Avenger: Age of Ultron': 6.0,
                                      'Django Unchained': 8.0,
                                      'Gone Girl': 7.0,
                                      'Horrible Bosses 2': 4.0,
                                      'Kill the Messenger': 6.5,
                                      'Zoolander': 4.0}}

movie_user_preferences['William']['Gone Girl']

data = []
for i in movie_user_preferences.keys():
    try:
        data.append((i
                     , movie_user_preferences[i]['Django Unchained']
                     , movie_user_preferences[i]['Avenger: Age of Ultron']))
    except:
        pass

df = pd.DataFrame(data=data, columns=['user', 'django', 'avenger'])
df

plt.scatter(df.django, df.avenger)
plt.xlabel('Django')
plt.ylabel('Avengers')
for i, txt in enumerate(df.user):
    plt.annotate(txt, (df.django[i], df.avenger[i]))
plt.show()

# Euclidean distance between Jill and Toby
sqrt(pow(8 - 7, 2) + pow(9 - 6, 2))

# Euclidean distance between Robert and Max
sqrt(pow(8 - 7, 2) + pow(7 - 7, 2))

# Similarity Score based on Euclidean distance between Jill and Toby
1 / (1 + sqrt(pow(8 - 7, 2) + pow(9 - 6, 2)))

# Similarity Score based on Euclidean distance between Robert and Max
1 / (1 + sqrt(pow(8 - 7, 2) + pow(7 - 7, 2)))


# Returns a distance-based similarity score for person1 and person2
def sim_distance(prefs, person1, person2):
    # Get the list of shared_items
    si = {}
    for item in prefs[person1]:
        if item in prefs[person2]:
            si[item] = 1

    # if they have no ratings in common, return 0
    if len(si) == 0: return 0

    # Add up the squares of all the differences
    sum_of_squares = sum([pow(prefs[person1][item] - prefs[person2][item], 2)
                          for item in prefs[person1] if item in prefs[person2]])

    return 1 / (1 + sum_of_squares)


sim_distance(movie_user_preferences, 'Sam', 'Toby')


def create_movie_user_df(input_data, user1, user2):
    data = []
    for movie in input_data[user1].keys():
        if movie in input_data[user2].keys():
            try:
                data.append((movie
                             , input_data[user1][movie]
                             , input_data[user2][movie]))
            except:
                pass

    return pd.DataFrame(data=data, columns=['movie', user1, user2])


df = create_movie_user_df(movie_user_preferences, 'Sam', 'William')
df

plt.scatter(df.Sam, df.William)
plt.xlabel('Sam')
plt.ylabel('William')
for i, txt in enumerate(df.movie):
    plt.annotate(txt, (df.Sam[i], df.William[i]))
plt.show()

pearsonr(df.Sam, df.William)

df = create_movie_user_df(movie_user_preferences, 'Sam', 'Julia')
df

plt.scatter(df.Sam, df.Julia)
plt.xlabel('Sam')
plt.ylabel('Julia')
for i, txt in enumerate(df.movie):
    plt.annotate(txt, (df.Sam[i], df.Julia[i]))
plt.show()

pearsonr(df.Sam, df.Julia)


# Returns the Pearson correlation coefficient for p1 and p2
def sim_pearson(prefs, p1, p2):
    # Get the list of mutually rated items
    si = {}
    for item in prefs[p1]:
        if item in prefs[p2]: si[item] = 1

    # Find the number of elements
    n = len(si)

    # if they are no ratings in common, return 0
    if n == 0: return 0

    # Add up all the preferences
    sum1 = sum([prefs[p1][it] for it in si])
    sum2 = sum([prefs[p2][it] for it in si])

    # Sum up the squares
    sum1Sq = sum([pow(prefs[p1][it], 2) for it in si])
    sum2Sq = sum([pow(prefs[p2][it], 2) for it in si])

    # Sum up the products
    pSum = sum([prefs[p1][it] * prefs[p2][it] for it in si])

    # Calculate Pearson score
    num = pSum - (sum1 * sum2 / n)
    den = sqrt((sum1Sq - pow(sum1, 2) / n) * (sum2Sq - pow(sum2, 2) / n))
    if den == 0: return 0

    r = num / den

    return r


sim_pearson(movie_user_preferences, 'Sam', 'Julia')


def topMatches(prefs, person, n=5, similarity=sim_pearson):
    scores = [(similarity(prefs, person, other), other)
              for other in prefs if other != person]

    # Sort the list so the highest scores appear at the top
    scores.sort()
    scores.reverse()
    return scores[0:n]


topMatches(movie_user_preferences, 'Toby', n=3, similarity=sim_distance)


# Gets recommendations for a person by using a weighted average
# of every other user's rankings
def getRecommendations(prefs, person, similarity=sim_pearson):
    totals = {}
    simSums = {}
    for other in prefs:
        # don't compare me to myself
        if other == person: continue
        sim = similarity(prefs, person, other)

        # ignore scores of zero or lower
        if sim <= 0: continue
        for item in prefs[other]:

            # only score movies I haven't seen yet
            if item not in prefs[person] or prefs[person][item] == 0:
                # Similarity * Score
                totals.setdefault(item, 0)
                totals[item] += prefs[other][item] * sim
                # Sum of similarities
                simSums.setdefault(item, 0)
                simSums[item] += sim

    # Create the normalized list
    rankings = [(total / simSums[item], item) for item, total in totals.items()]

    # Return the sorted list
    rankings.sort()
    rankings.reverse()
    return rankings


getRecommendations(movie_user_preferences, 'Toby')

getRecommendations(movie_user_preferences, 'Toby', similarity=sim_distance)


def transformPrefs(prefs):
    result = {}
    for person in prefs:
        for item in prefs[person]:
            result.setdefault(item, {})

            # Flip item and person
            result[item][person] = prefs[person][item]
    return result


transformPrefs(movie_user_preferences)


def calculateSimilarItems(prefs, n=10):
    # Create a dictionary of items showing which other items they
    # are most similar to.
    result = {}

    # Invert the preference matrix to be item-centric
    itemPrefs = transformPrefs(prefs)
    c = 0
    for item in itemPrefs:
        # Status updates for large datasets
        c += 1
        if c % 100 == 0: print
        "%d / %d" % (c, len(itemPrefs))
        # Find the most similar items to this one
        scores = topMatches(itemPrefs, item, n=n, similarity=sim_distance)
        result[item] = scores
    return result


itemsim = calculateSimilarItems(movie_user_preferences)
itemsim


def getRecommendedItems(prefs, itemMatch, user):
    userRatings = prefs[user]
    scores = {}
    totalSim = {}

    # Loop over items rated by this user
    for (item, rating) in userRatings.items():

        # Loop over items similar to this one
        for (similarity, item2) in itemMatch[item]:

            # Ignore if this user has already rated this item
            if item2 in userRatings: continue

            # Weighted sum of rating times similarity
            scores.setdefault(item2, 0)
            scores[item2] += similarity * rating

            # Sum of all the similarities
            totalSim.setdefault(item2, 0)
            totalSim[item2] += similarity

    # Divide each total score by total weighting to get an average
    rankings = [(score / totalSim[item], item) for item, score in scores.items()]

    # Return the rankings from highest to lowest
    rankings.sort()
    rankings.reverse()
    return rankings


getRecommendedItems(movie_user_preferences, itemsim, 'Toby')
